# data kilder
Data for danmark: https://www.statbank.dk/20050 

Data for Norge: https://www.ssb.no/en/statbank/table/03672/tableViewLayout1/ 

In [211]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

df_den = pd.read_excel('denmark1991_2005.xlsx')
df_no = pd.read_excel('norge.xlsx')
df_den2 = pd.read_excel('denmark2006_2009.xlsx')


In [212]:
df_den.head(15)

,"Hospital patients by dominant diagnosis, region, age and time",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,Units: Number,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0
2,"DISEASES OF THE RESPIRATORY ORGANS, TOTAL",All Denmark,"Age, total",43558.0,44474.0,48448.0,44541.0,48288.0,48241.0,48305.0,48996.0,49931.0,47902.0,49226.0,48016.0,50268.0,48864.0,50442.0
3,NaN,NaN,0 years,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,NaN,1-4 years,8736.0,8902.0,10027.0,8861.0,10177.0,9955.0,10016.0,9764.0,10049.0,9265.0,10505.0,9056.0,9874.0,9358.0,10194.0
5,NaN,NaN,5-9 years,3397.0,3442.0,3769.0,3176.0,3658.0,3434.0,3393.0,3400.0,3398.0,3070.0,3572.0,3209.0,3491.0,3082.0,3163.0
6,NaN,NaN,10-14 years,1706.0,1744.0,1597.0,1396.0,1420.0,1239.0,1311.0,1225.0,1236.0,1212.0,1317.0,1198.0,1401.0,1281.0,1373.0
7,NaN,NaN,15-19 years,2949.0,2576.0,2579.0,2359.0,2300.0,2088.0,1955.0,1832.0,1807.0,1682.0,1655.0,1517.0,1671.0,1595.0,1668.0
8,NaN,NaN,20-24 years,1822.0,1721.0,1793.0,1735.0,1810.0,1663.0,1630.0,1485.0,1425.0,1376.0,1290.0,1304.0,1265.0,1166.0,1102.0
9,NaN,NaN,25-29 years,1651.0,1600.0,1605.0,1642.0,1498.0,1548.0,1442.0,1471.0,1347.0,1420.0,1254.0,1331.0,1329.0,1162.0,1141.0


In [213]:
df_den_clean = df_den.iloc[1:].copy()
# First, select only the columns we want (column 2 onwards - age groups + year data)
df_den_clean = df_den_clean.iloc[:, 2:]
# Now set the column names: keep the first column name (age groups) and use years from row 0
new_columns = [df_den_clean.columns[0]] + list(df_den_clean.iloc[0, 1:])
df_den_clean.columns = new_columns
df_den_clean = df_den_clean.iloc[1:]  # Remove the row with years 
df_den_clean.reset_index(drop=True, inplace=True)
df_den_clean = df_den_clean.loc[:, df_den_clean.columns.notna()]

df_den_clean.head(15)
df_den_filtered = df_den_clean.iloc[[2, 3, 4, 5]].copy()
df_den_filtered.reset_index(drop=True, inplace=True)

df_den_filtered.head()

,Unnamed: 2,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0
0,1-4 years,8736.0,8902.0,10027.0,8861.0,10177.0,9955.0,10016.0,9764.0,10049.0,9265.0,10505.0,9056.0,9874.0,9358.0,10194.0
1,5-9 years,3397.0,3442.0,3769.0,3176.0,3658.0,3434.0,3393.0,3400.0,3398.0,3070.0,3572.0,3209.0,3491.0,3082.0,3163.0
2,10-14 years,1706.0,1744.0,1597.0,1396.0,1420.0,1239.0,1311.0,1225.0,1236.0,1212.0,1317.0,1198.0,1401.0,1281.0,1373.0
3,15-19 years,2949.0,2576.0,2579.0,2359.0,2300.0,2088.0,1955.0,1832.0,1807.0,1682.0,1655.0,1517.0,1671.0,1595.0,1668.0


In [214]:
df_den2_clean = df_den2.iloc[1:].copy()
df_den2_clean = df_den2_clean.iloc[:, 2:]
new_columns = [df_den2_clean.columns[0]] + list(df_den2_clean.iloc[0, 1:])
df_den2_clean.columns = new_columns
df_den2_clean = df_den2_clean.iloc[1:5] 
df_den2_clean.reset_index(drop=True, inplace=True)

df_den2_clean.head(15)

,Unnamed: 2,2006.0,2007.0,2008.0,2009.0
0,1-4 years,9804.0,10085.0,8783.0,10018.0
1,5-9 years,2954.0,2984.0,2280.0,2788.0
2,10-14 years,1384.0,1394.0,1144.0,1234.0
3,15-19 years,1776.0,1822.0,1582.0,1851.0


In [215]:
df_den_combined = pd.merge(df_den_filtered, df_den2_clean, on=df_den_filtered.columns[0], how='left')
df_den_combined.head()

,Unnamed: 2,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0
0,1-4 years,8736.0,8902.0,10027.0,8861.0,10177.0,9955.0,10016.0,9764.0,10049.0,9265.0,10505.0,9056.0,9874.0,9358.0,10194.0,9804.0,10085.0,8783.0,10018.0
1,5-9 years,3397.0,3442.0,3769.0,3176.0,3658.0,3434.0,3393.0,3400.0,3398.0,3070.0,3572.0,3209.0,3491.0,3082.0,3163.0,2954.0,2984.0,2280.0,2788.0
2,10-14 years,1706.0,1744.0,1597.0,1396.0,1420.0,1239.0,1311.0,1225.0,1236.0,1212.0,1317.0,1198.0,1401.0,1281.0,1373.0,1384.0,1394.0,1144.0,1234.0
3,15-19 years,2949.0,2576.0,2579.0,2359.0,2300.0,2088.0,1955.0,1832.0,1807.0,1682.0,1655.0,1517.0,1671.0,1595.0,1668.0,1776.0,1822.0,1582.0,1851.0


In [216]:
df_den_combined = df_den_combined.iloc[:, [0] + list(range(9, 19))]
# Add rows 1+2 (0-4 years + 5-9 years = 0-9 years)
row_0_9 = df_den_combined.iloc[0] + df_den_combined.iloc[1]
row_0_9.iloc[0] = '0-9 years'  # Set the age group name

# Add rows 2+3 (10-14 years + 15-19 years = 10-19 years) 
row_10_19 = df_den_combined.iloc[2] + df_den_combined.iloc[3]
row_10_19.iloc[0] = '10-19 years'  # Set the age group name

# Create new DataFrame with the combined rows
df_den_final = pd.DataFrame([row_0_9, row_10_19])
df_den_final.reset_index(drop=True, inplace=True)
df_den_final.columns.values[0] = 'Age Group'
df_den_final.head()

df_den_final.to_excel('denmark_final.xlsx', index=False)

In [217]:
df_no_clean = df_no.iloc[2:].copy()
df_no_clean = df_no_clean.iloc[:, 3:]
new_columns = [df_no_clean.columns[0]] + list(df_no_clean.iloc[0, 1:])
df_no_clean.columns = new_columns
df_no_clean = df_no_clean.iloc[2:4]
df_no_clean.reset_index(drop=True, inplace=True)
df_no_clean.columns.values[0] = 'Age Group'
df_no_clean.head(15)
df_no_clean.to_excel('norway_final.xlsx', index=False)

In [218]:
df_royk = pd.read_excel('røykere99_08.xlsx')

In [219]:
df_royk.head(15)

,"05307: Dagligrøykere og av-og-til-røykere (16-79 år), etter kjønn, alder, statistikkvariabel og år",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Antall respondenter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,1999,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0
3,Begge kjønn,I alt,5104,5208.0,5216.0,5485.0,5288.0,5243.0,5225.0,4968.0,4997.0,4637.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,I 2024 ble for første gang undersøkelsen gjenn...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Treårig glidende gjennomsnitt fram til 1991. F...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Sist endret:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Antall respondenter:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_royk_clean = df_royk.iloc[2:].copy()
# First, select only the columns we want (column 2 onwards - age groups + year data)
df_royk_clean = df_royk_clean.iloc[:, 1:]
# Now set the column names: keep the first column name (age groups) and use years from row 0
new_columns = [df_royk_clean.columns[0]] + list(df_royk_clean.iloc[0, 1:])
df_royk_clean.columns = new_columns
df_royk_clean = df_royk_clean.iloc[1:]  # Remove the row with years
df_royk_clean.reset_index(drop=True, inplace=True)
df_royk_clean = df_royk_clean.loc[:, df_royk_clean.columns.notna()]

df_royk_clean.head(15)
df_royk_filtered = df_royk_clean.iloc[[0]].copy()
df_royk_filtered.reset_index(drop=True, inplace=True)

df_royk_filtered.head()


,Unnamed: 1,1999,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0
0,I alt,5104,5208.0,5216.0,5485.0,5288.0,5243.0,5225.0,4968.0,4997.0,4637.0


In [245]:
df_royk_den = pd.read_excel('røykere_den_raw_99_08.xlsx')

df_royk_den_clean = df_royk_den.iloc[1:].copy()
df_royk_den_clean.columns = new_columns
df_royk_den_clean = df_royk_den_clean.iloc[1:2]  # Remove the row with years
df_royk_den_clean.reset_index(drop=True, inplace=True)
df_royk_den_clean = df_royk_den_clean.loc[:, df_royk_den_clean.columns.notna()]
df_royk_filtered = df_royk_den_clean.iloc[[0]].copy()
df_royk_filtered.reset_index(drop=True, inplace=True)

df_royk_filtered.head()
df_royk_filtered.to_excel('røykere_filtered_den.xlsx', index=False)
